<a target="_blank" href="https://colab.research.google.com/github/umanitoba-meagher-projects/public-experiments/blob/main/jupyter-notebooks/Visualize%20Image%20Information/sightings-bar-graph.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
"""
Author: Ryleigh J. Bruce
Date: June 11, 2024

Purpose: To create several bar graphs depicting the volume of animal sightings per location across, months, seasons, and species.


Note: The author generated this text in part with GPT-4,
OpenAI’s large-scale language-generation model. Upon generating
draft code, the authors reviewed, edited, and revised the code
to their own liking and takes ultimate responsibility for
the content of this code.

"""

# Introduction

This notebook provides a systematic approach to visualizing animal sighting data across various locations, months, seasons, and species. The primary objective is to transform raw observational data into interpretable bar graphs that reveal patterns in animal presence and distribution. The workflow begins by downloading the dataset from the Borealis repository using their API, followed by importing essential Python libraries such as pandas and matplotlib for data manipulation and visualization. The dataset, which is structured as an Excel file, is loaded into a DataFrame, and preliminary data checks are performed to ensure integrity. The methodology involves extracting temporal features (such as month and season) from date fields, aggregating sighting counts by relevant categories (location, month, season, species), and generating grouped bar plots to illustrate these aggregations. Each section of the notebook is modular, focusing on a specific aspect of the data—monthly, seasonal, and species-based analyses—using consistent data processing and plotting techniques. The notebook emphasizes reproducibility and clarity, with markdown explanations accompanying each code block to guide users through the logic and purpose of each step. The benefits of this approach include the ability to quickly identify spatial and temporal trends in animal sightings, support for further ecological or spatial analysis, and adaptability for use with different datasets or research questions. The notebook is structured to be accessible for users with varying levels of experience in Python and data analysis, serving both as a practical tool for data exploration and as an educational resource for understanding the application of Python in ecological data visualization.

# Critical Uses & Adaptability

## What the Notebooks Can Be Used For:

- **Dataset Exploration:**
  - Enables users to examine the distribution and frequency of animal sightings across different spatial and temporal dimensions. The notebook's aggregation and plotting steps (see code blocks for grouping and plotting) facilitate the identification of trends and anomalies in the dataset.

- **Educational Purposes & Demonstrations:**
  - Serves as a resource for demonstrating the use of Python code, scripting, and basic data analysis techniques in the context of ecological or image-based datasets. The notebook includes step-by-step explanations and code for data loading, transformation, and visualization, making it suitable for instructional settings or self-guided learning. It also introduces foundational concepts relevant to machine learning workflows, such as feature extraction and data aggregation.

- **Feature Extraction:**
  - Illustrates how to derive new variables (e.g., month, season, total sightings) from raw data fields. These derived features are used for further analysis and visualization, as shown in the code blocks that create new DataFrame columns and perform groupby operations.

## How the Notebook Can Be Adapted:

- **Integration with Spatial Design & Architectural Studies:**
  - The notebook's approach to aggregating and visualizing spatial data can be applied to site analysis tasks in spatial design or architecture. By swapping the animal sighting dataset with spatial or architectural datasets, users can analyze patterns such as foot traffic, environmental features, or usage intensity across locations and time periods.

- **Variables & Customization:**
  - Variables such as 'locationID', 'Date', 'SpeciesList', and count columns are explicitly defined and can be modified to match the schema of alternative datasets. Users can adjust aggregation keys or add new features as needed for their specific analysis goals.

- **Swapping Datasets:**
  - The notebook is designed to accommodate different datasets by changing the file path and ensuring the new dataset contains compatible columns. The code block responsible for loading the dataset (see the cell beginning with `file_path = ...` and `df = pd.read_excel(file_path)`) can be edited to point to a custom dataset, provided the necessary columns are present or appropriately renamed.

- **Scalability:**
  - The modular structure of the notebook allows for the addition of new analysis sections or the scaling up of existing ones to handle larger datasets or more complex grouping variables. The use of pandas and matplotlib ensures compatibility with large data volumes and supports further extension with additional Python libraries if required.

## Module: Borealis Data Repository Integration

Here the Borealis API is queried to retrieve and download the dataset (combined_animals.xlsx) from the public repository. Utility functions are defined for checking, downloading, and unzipping files as needed. Additionally, we download the image data that will be used to train our machine learning model. The data is hosted in the University of Manitoba Dataverse (https://borealisdata.ca/dataverse/manitoba), a research data repository. The images used in this notebook were collected as part of the 'Understanding Animals' project at University of Manitoba Faculty of Architecture, online at Wild Winnipeg and Teaching with Images.

In [ ]:
# Borealis API configuration
import requests
import zipfile
import os
import shutil
import pandas as pd

BOREALIS_SERVER = "https://borealisdata.ca"

def get_public_dataset_info(persistent_id):
    """
    Get information about a public dataset
    """
    url = f"{BOREALIS_SERVER}/api/datasets/:persistentId/"
    params = {"persistentId": persistent_id}

    response = requests.get(url, params=params)

    if response.status_code == 200:
        dataset_info = response.json()
    else:
        print(f"Cannot access dataset: {response.status_code}")
        return None

    """
    Get a list of files in a public dataset
    """
    # Access the list of files from the dataset_info dictionary
    files_list = dataset_info['data']['latestVersion']['files']

    # Create an empty list to store file information
    file_info_list = []

    # Iterate through the files list and append file ID and filename to the list
    for file_info in files_list:
        file_id = file_info['dataFile']['id']
        filename = file_info['dataFile']['filename']
        file_info_list.append({"file_id": file_id, "filename": filename})

    return file_info_list

def download_public_file(file_id, filename=None, save_path="./"):
    """
    Download a specific public file from a dataset by its file ID
    No authentication required
    """
    url = f"{BOREALIS_SERVER}/api/access/datafile/{file_id}"

    response = requests.get(url, stream=True)

    if response.status_code == 200:
        # Use provided filename or try to determine from headers
        if not filename:
            if "Content-Disposition" in response.headers:
                cd = response.headers["Content-Disposition"]
                if "filename=" in cd:
                    filename = cd.split("filename=")[1].strip('"')

            # Fallback to file ID if no filename found
            if not filename:
                filename = str(file_id)

        file_path = os.path.join(save_path, filename)

        with open(file_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        print(f"SUCCESS: File downloaded to {file_path}")
        return file_path
    else:
        print(f"ERROR: {response.status_code}: File may be restricted or not found")
        return None

def is_zip_file(filepath):
    """
    Checks if a file is a valid zip file.
    """
    try:
        return zipfile.is_zipfile(filepath)
    except:
        return False

def unzip_file(filepath, extract_path="./"):
    """
    Unzips a zip file to a specified path and returns the name of the top-level extracted folder.
    Returns None if not a zip file or extraction fails.
    """
    if is_zip_file(filepath):
        try:
            with zipfile.ZipFile(filepath, 'r') as zip_ref:
                # Get the name of the top-level directory within the zip
                top_level_folder = None
                for file_info in zip_ref.infolist():
                    parts = file_info.filename.split('/')
                    if parts[0] and len(parts) > 1:
                        top_level_folder = parts[0]
                        break

                zip_ref.extractall(extract_path)
                print(f"SUCCESS: Successfully unzipped {filepath} to {extract_path}")
                return top_level_folder

        except Exception as e:
            print(f"ERROR: Error unzipping {filepath}: {e}")
            return None
    else:
        print(f"INFO: {filepath} is not a valid zip file.")
        return None

# Initialize and download the dataset
public_doi = "doi:10.5683/SP3/H3HGWF"
print("Initializing Borealis dataset access...")

# Get dataset information and file list
file_info_list = get_public_dataset_info(public_doi)
file_path = None

if file_info_list:
    print(f"\nFound {len(file_info_list)} files in the dataset:")
    for i, file_info in enumerate(file_info_list):
        print(f"{i+1}. ID: {file_info['file_id']}, Filename: {file_info['filename']}")

    # Look specifically for the combined_animals.xlsx file
    target_file = None
    for file_info in file_info_list:
        if file_info['filename'] == 'combined_animals.xlsx':
            target_file = file_info
            break

    if target_file:
        print(f"\nFound target file: {target_file['filename']}")

        # Download with the correct filename
        downloaded_path = download_public_file(target_file['file_id'], target_file['filename'])

        if downloaded_path:
            file_path = downloaded_path
            print(f"Excel file ready at: {file_path}")

            # Verify it's a valid Excel file
            try:
                df_test = pd.read_excel(file_path)
                print(f"SUCCESS: File verified! Shape: {df_test.shape}")
                print("Columns:", list(df_test.columns)[:10])  # Show first 10 columns
                del df_test  # Clean up test dataframe
            except Exception as e:
                print(f"WARNING: Could not read as Excel file: {e}")

                # Check if it might be a zip file that was misnamed
                if is_zip_file(file_path):
                    print("File appears to be a zip archive, extracting...")
                    extracted_folder = unzip_file(file_path)

                    if extracted_folder:
                        # Look for Excel files in extracted folder
                        for root, dirs, files in os.walk(extracted_folder):
                            for file in files:
                                if file.endswith(('.xlsx', '.xls')):
                                    excel_path = os.path.join(root, file)
                                    print(f"Found Excel file in archive: {excel_path}")
                                    file_path = excel_path
                                    break
        else:
            print("Download failed")
    else:
        print("\nTarget file 'combined_animals.xlsx' not found in dataset")
        print("Available files:")
        for file_info in file_info_list:
            print(f"- {file_info['filename']}")
else:
    print("Could not retrieve dataset information from Borealis")

# Final check and setup
if file_path and os.path.exists(file_path):
    print(f"\nFINAL: Dataset ready at: {file_path}")
    print("You can now run your pandas code to load the data!")
else:
    print("\nERROR: Could not successfully download and prepare the dataset")

    # Show what files are actually in the directory
    print("\nFiles currently in directory:")
    for item in os.listdir('.'):
        if os.path.isfile(item):
            size = os.path.getsize(item)
            print(f"- {item} ({size} bytes)")

print("\nBorealis dataset access complete.")

# Volume of Sightings Across Locations Per Month

## Module: Importing the Necessary Libraries and Loading them into a DataFrame

Here the `pandas` and `matplotlib.pyplot` libraries are being imported in order to use their data manipulation and visualization properties later on in the script. `pandas` will be particularly useful for transforming the data found within the given spreadsheet into an easier to manipulate DataFrame format, and `matplotlib.pyplot` is crucial for generating the graph.

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt

The file path (combined_animals.xlsx) is set to the dataset retrieved via Borealis. The Excel file is then loaded into a DataFrame using `pandas`.

In [ ]:
# Read the Excel file
file_path = 'combined_animals.xlsx'  # Change this to the correct file path if necessary
df = pd.read_excel(file_path)

The `print(df.head())` code returns the first five rows of the DataFrame in order to view the contained data and ensure the file has been properly transformed into a DataFrame.

In [ ]:
# Check the first few rows of the DataFrame to ensure it's loaded correctly
print(df.head())

Here the `pd.to_datetime(df[‘Date’])` function extracts the information from the ‘Date’ column within the DataFrame and turns that data into datetime format. This conversion is critical for performing accurate time-based calculations.

In [ ]:
# Ensure the Date column is treated as a datetime object
df['Date'] = pd.to_datetime(df['Date'])

## Module: Manipulating the Dataframe

The `.strftime(‘%B’)` method is used to specify the format of the datetime information from the 'Date' column. The `%B` code returns the full name of the month.

`df[‘Month’]` creates a new column within the DataFrame called ‘Month’, which stores the formatted information extracted from the ‘Date’ column.

In [ ]:
# Extract the month from the Date column
df['Month'] = df['Date'].dt.strftime('%B')

Using `df` to refer to the data stored within the DataFrame, this code accesses the information in the AdultCount and JuvenileCount columns, and uses `df[‘AdultCount’] + [‘JuvenileCount’]` to calculate the value of `df[‘TotalSightings’]`. This information is stored within the DataFrame as a new column called ‘TotalSightings’.

In [ ]:
# Calculate the total number of animals spotted (AdultCount + JuvenileCount)
df['TotalSightings'] = df['AdultCount'] + df['JuvenileCount']

Here the DataFrame is grouped by the ‘locationID’ and ‘Month’ columns, ensuring that separate sums will be calculated for each unique combination of location ID and month. The `sum` operation is then used to calculate the sum of each ‘TotalSightings’ column per unique grouping. A new DataFrame called `aggregated_data` is then created to store the new data.

In [ ]:
# Aggregate the data to sum the total sightings per month and location
aggregated_data = df.groupby(['locationID', 'Month']).agg({'TotalSightings': 'sum'}).reset_index()

## Module: Plotting the Bar Graph

This code begins by creating an empty figure and set of subplots, `fig, ax`, while `figsize=(14, 8)` sets the dimensions of the figure in inches.




Then the code retrieves all of the unique month values from the DataFrame. The `for month in aggregated_data[‘Month’].unique():` iterates over each unique month, while `subset = aggregated_data[aggregated_data[‘Month’] == month]` filters the information so that only rows matching the current month of the loop is included in the creation of that month’s subset.

`ax.bar()` creates a bar plot for the given subset, and the x-axis values and height of the bars are set as ‘locationID’ and ‘TotalSightings’ respectively.

Next labels are assigned to the x and y axes, and a title is provided for the entire graph. `ax.legend(title=’Month’)` generates a title for the legend.

Finally, the graph is visualized using `plt.show()`.

In [ ]:
# Generate a bar plot
fig, ax = plt.subplots(figsize=(14, 8))

# Create a bar plot for each month and location ID
for month in aggregated_data['Month'].unique():
    subset = aggregated_data[aggregated_data['Month'] == month]
    ax.bar(subset['locationID'], subset['TotalSightings'], label=month)

# Set the labels and title
ax.set_xlabel('Location ID')
ax.set_ylabel('Number of Sightings')
ax.set_title('Volume of Species Sightings by Location ID and Month')
ax.legend(title='Month')

ax.set_xticks(aggregated_data['locationID'].unique())
ax.set_xticklabels(aggregated_data['locationID'].unique(), rotation=90)

plt.xticks(rotation=90)

# Display the plot
plt.show()

# Volume of Sightings Across Locations Per Season

In order to create a bar graph depicting the volume of sightings across locations by season, the first few blocks of code will remain the same as in the previous script.

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt

# Read the Excel file
file_path = './combined_animals.xlsx'
df = pd.read_excel(file_path)

# Ensure the Date column is treated as a datetime object
df['Date'] = pd.to_datetime(df['Date'])

# Extract the month from the Date column
df['Month'] = df['Date'].dt.month

## Module: Manipulating the Dataframe

A dictionary is created and assigned to the `month_to_season` variable. The dictionary stores the months of the year in integer format and their corresponding seasons.

In [ ]:
# Map the months to seasons
month_to_season = {3: 'Spring', 4: 'Spring', 5: 'Spring',
                   6: 'Summer', 7: 'Summer', 8: 'Summer',
                   9: 'Autumn', 10: 'Autumn', 11: 'Autumn',
                   12: 'Winter', 1: 'Winter', 2: 'Winter'}

A new column is created in the DataFrame called ‘Season’ and the script populates the column with the corresponding value for each month in the ‘Month’ column of the DataFrame.

In [ ]:
df['Season'] = df['Month'].map(month_to_season)

Before calculating totals, missing values in `AdultCount` and `JuvenileCount` are filled with zero to ensure accurate aggregation. The calculation of the ‘TotalSightings’ variable, creation of the `aggregated_data` DataFrame, and figure generation are all the same as the previous bar graph script.

In [ ]:
# Fill NaN values with 0 for count columns
df['AdultCount'] = df['AdultCount'].fillna(0)
df['JuvenileCount'] = df['JuvenileCount'].fillna(0)

# Calculate the total number of animals spotted (AdultCount + JuvenileCount)
df['TotalSightings'] = df['AdultCount'] + df['JuvenileCount']

# Aggregate the data to sum the total sightings per month and location
aggregated_data = df.groupby(['locationID', 'Season']).agg({'TotalSightings': 'sum'}).reset_index()

# Generate a bar plot
fig, ax = plt.subplots(figsize=(14, 8))

# Get the colormap
color_map = plt.get_cmap('tab20c')

# Map each season to a color from the colormap
season_to_color = {'Spring': color_map(0), 'Summer': color_map(4), 'Autumn': color_map(8), 'Winter': color_map(12)}

# Create a bar plot for each season and location ID
for season in aggregated_data['Season'].unique():
    subset = aggregated_data[aggregated_data['Season'] == season]
    ax.bar(subset['locationID'], subset['TotalSightings'], label=season, color=season_to_color[season])

ax.set_xticks(aggregated_data['locationID'].unique())
ax.set_xticklabels(aggregated_data['locationID'].unique(), rotation=90)

# Set the labels and title
ax.set_xlabel('Location ID')
ax.set_ylabel('Number of Sightings')
ax.set_title('Volume of Species Sightings by Location ID and Season')
ax.legend(title='Season')

# Make sure all X labels are displayed
plt.xticks(rotation=90)

# Display the plot
plt.show()

## Module: Plotting the Graph

Here the `plt.get_cmap()` function is used to import the ‘tab20c’ color map. Colors from the imported color map are then assigned to each season.

In [ ]:
# Get the colormap
color_map = plt.get_cmap('tab20c')

# Map each season to a color from the colormap
season_to_color = {'Spring': color_map(0), 'Summer': color_map(4), 'Autumn': color_map(8), 'Winter': color_map(12)}

The season information within the `aggregated_data` DataFrame is iterated over and filtered, just as the month data was in the previous script. The x-axis values, bar heights, and labels are also determined in the same way.

In [ ]:
# Create a bar plot for each season and location ID
for season in aggregated_data['Season'].unique():
    subset = aggregated_data[aggregated_data['Season'] == season]
    ax.bar(subset['locationID'], subset['TotalSightings'], label=season, color=season_to_color[season])

This code block simply formats the x-ticks and x-tick labels appropriately for the graph. `rotation=90` rotates the x-tick labels by 90 degrees to maintain legibility.


In [ ]:
ax.set_xticks(aggregated_data['locationID'].unique())
ax.set_xticklabels(aggregated_data['locationID'].unique(), rotation=90)

Here the labels for each axis are set, as well as the title for the overall graph. `plt.xticks(rotation=90)` ensures that all of the x-tick labels will be displayed, and the plot is visualized using `plt.show()`.

In [ ]:
# Create a new figure and axis for the seasonal plot
fig, ax = plt.subplots(figsize=(14, 8))

# Create a bar plot for each season and location ID
for season in aggregated_data['Season'].unique():
    subset = aggregated_data[aggregated_data['Season'] == season]
    ax.bar(subset['locationID'], subset['TotalSightings'],
           label=season, color=season_to_color[season])

# Set the labels and title
ax.set_xlabel('Location ID')
ax.set_ylabel('Number of Sightings')
ax.set_title('Volume of Species Sightings by Location ID and Season')
ax.legend(title='Season')

# Ensure all X labels are shown
ax.set_xticks(aggregated_data['locationID'].unique())
ax.set_xticklabels(aggregated_data['locationID'].unique(), rotation=90)

plt.tight_layout()
plt.show()

# Volume of Sightings Across Locations Per Species

In order to plot a bar graph to depict the volume of sightings per species across locations, the same libraries must be imported with the addition of the `numpy` library. The `numpy` library is crucial for working with arrays and large data.

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Just like the previous two scripts, the Excel sheet must be converted into a two-dimensional DataFrame before any data manipulation can take place.


In [ ]:
# Read the Excel file
file_path = './combined_animals.xlsx'  # Change this to the correct file path if necessary
df = pd.read_excel(file_path)

## Module: Manipulating the Dataframe

The species information does not need to be converted into any other formats before the graph is generated, so the manipulation of the DataFrame will be much simpler for this script.

The ‘TotalSightings’ column will be added to the DataFrame after the values from the ‘AdultCount’ and ‘JuvenileCount’ columns have been added together. Then the rows will be grouped by locationID and SpeciesList, creating a new DataFrame called `aggregated_data`. The number of unique species is determined using `.nunique()`, which is then used to evenly assign colors from the colormap.

In [ ]:
# Ensure the Date column is treated as a datetime object
df['Date'] = pd.to_datetime(df['Date'])

# Calculate the total number of animals spotted (AdultCount + JuvenileCount)
df['TotalSightings'] = df['AdultCount'] + df['JuvenileCount']

# Aggregate the data to sum the total sightings per species and location
aggregated_data = df.groupby(['locationID', 'SpeciesList']).agg({'TotalSightings': 'sum'}).reset_index()

# Determine the number of unique species (needed for colors)
num_categories = len(aggregated_data['SpeciesList'].unique())

## Module: Plotting the Graph

The tab20c colormap is imported using the `plt.get_cmap()` function, and `np.linspace(0, 1, num_categories)` creates an equally spaced list of `num_categories` between 0 and 1 in order to evenly spread the colors selected for each species. A dictionary called `species_to_color` is then created to map a color to each species. The final code block ensures that there is a bar plot for each individual species. The graph is plotted in the same way as the previous two scripts, with a small difference to ensure the legend does not clip the graph. `plt.tight_layout(rect=[0, 0, 0.85, 1])` ensures that there is enough space left for the legend.

In [ ]:
# Set up the figure and axis for plotting
fig, ax = plt.subplots(figsize=(14, 8))

# Create color map
colors = plt.get_cmap('tab20c')(np.linspace(0, 1, num_categories))

# Generate a mapping of SpeciesList to colors
species_to_color = {
    species: colors[i]
    for i, species in enumerate(sorted(aggregated_data['SpeciesList'].unique()))
}

# Iterate over each species to plot
for species, group in aggregated_data.groupby('SpeciesList'):
    ax.bar(group['locationID'], group['TotalSightings'],
           label=species, color=species_to_color[species])

# Set the labels and title
ax.set_xlabel('Location ID')
ax.set_ylabel('Number of Sightings')
ax.set_title('Volume of Species Sightings by Location ID and Species')
ax.legend(title='Species', bbox_to_anchor=(1.05, 1), loc='upper left')

# Ensure x-axis shows every location ID
ax.set_xticks(aggregated_data['locationID'].unique())
ax.set_xticklabels(aggregated_data['locationID'].unique(), rotation=90)

# Improve layout and avoid clipping
plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.show()
